# Mal Webscraping

## Import Libraries

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np

import datetime
import time

import itertools

## Making fuctipon for more detailed information

<br> this info is found each anime's own webpage, thus it takes long

such as:
- anime type (TV/Movie/OVA)
- premaried (season and year)
- broadcast (day of week)
- stuio responsible
- duration
- age rating
- producers
- main source

In [ ]:
# response gives doctype: html, and PUBLIC
# detail_url = f"https://myanimelist.net/anime/"
# response = requests.get(detail_url).text
# soup = BeautifulSoup(response, 'html.parser')


def more_detail(mal_ref):
      genreArray = []
      genreArray =[]


      url_Ref = f"https://myanimelist.net/anime/{mal_ref}"

      responseRef = requests.get(url_Ref).text
      soupRef = BeautifulSoup(responseRef, 'html.parser')

      time.sleep(2)

      typeAni      = (soupRef.select_one('span:-soup-contains("Type:")')).find_next().text

      if typeAni == 'TV':
            # status      = (soupRef.select_one('span:-soup-contains("Status:")').find_next_sibling(string=True).split('  ')[1].split('\n')[0])
            premiered  =   (soupRef.select_one('span:-soup-contains("Premiered:")').find_next_sibling().text)
            broadcast   = (soupRef.select_one('span:-soup-contains("Broadcast:")').find_next_sibling(string=True).split(' '*4)[1].split(' at ')[0])
     
            duration    = (soupRef.select_one('span:-soup-contains("Duration:")').find_next_sibling(string=True).split('  ')[1].split(' min. per')[0])
            # ageRating      = (soupRef.select_one('span:-soup-contains("Rating:")').find_next_sibling(string=True).split(' ')[2])


            # return typeAni, status, premiered, broadcast, producers,studio, source, duration,rating
      
      else:
            # status      = (soupRef.select_one('span:-soup-contains("Status:")').find_next_sibling(string=True).split('  ')[1].split('\n')[0])
  
            premiered   = np.nan
            broadcast   = np.nan
                 
            duration    = (soupRef.select_one('span:-soup-contains("Duration:")').find_next_sibling(string=True)).split('  ')[1].split('.\n')[0]



      status      = (soupRef.select_one('span:-soup-contains("Status:")').find_next_sibling(string=True).split('  ')[1].split('\n')[0])
      producers   = (soupRef.select_one('span:-soup-contains("Producers:")').find_next_sibling().text) ## Not done
      studio      = (soupRef.select_one('span:-soup-contains("Studios:")').find_next_sibling().text)
      source      = (soupRef.select_one('span:-soup-contains("Source:")').find_next_sibling(string=True).split('  ')[1].split('\n')[0])
      ageRating      = (soupRef.select_one('span:-soup-contains("Rating:")').find_next_sibling(string=True).split(' ')[2])


      # genres = soupRef.select_one('span:-soup-contains("Genres:")')#.find_next_siblings('span', attrs={'itemprop':'genre'})

      #-------------------------
      gerne = (soupRef.select_one('span:-soup-contains("Genres:")'))
      
      try:
          genres = gerne.find_next_siblings()
          for i in genres:
              genreArray.append(i.text)

      except AttributeError:
      #     print('One Genre')
          genreArray.append('Fixing_one_Genre')
      #-------------------------




      return status, premiered, broadcast, producers,studio, source, duration, ageRating, genreArray
      # return genreArray


      # * For TV show all
      # return status, premiered ,broadcast, producers,studio #,source, duration,rating


      #* For Movies and other
      # return status, producers, studio, source, duration, rating

      #* For testing:
      # return typeAni

# more_detail(39486)

## Main webscraping function

Scrapes the data from the top anime main page

scrapes data like:
- 

In [ ]:
titleListnew = []
ratingListnew = []
typeListnew = []
startDateListnew = []
endDateListnew = []
episodeListnew = []
membersListnew = []
refenceListnew = []

statusList = []
premieredList = []
broadcastList = []
producerList = []
studioList = []
sourceList = []
durationList = []
ageRatingList = []
genreList = []

## 50 entries takes apprx -- 3m45s
## 310 entries takes apprx -- 22m01s

### Last score stuff at page: 13 750
for page in range(0,200,50):
    pages = f'https://myanimelist.net/topanime.php?limit={page}'

    newResponse = requests.get(pages).text
    soup = BeautifulSoup(newResponse, 'html.parser')

    time.sleep(10)

    
    commoon_link = soup.find_all('tr', class_ = 'ranking-list')
    
    for elemment in commoon_link:
        name = elemment.find('h3', attrs={'class':'hoverinfo_trigger fl-l fs14 fw-b anime_ranking_h3'})
        score = elemment.find(class_='score ac fs14').text.split('\n')[0]
        
        # # Have to deconstruct
        aniInfo = elemment.find(class_ = 'information di-ib mt4').text.split(' '*8)
        aniType = aniInfo[1].split(' ')[0]
        members = aniInfo[3].split(' ')[0].replace(',', '')
        
        epiTotal = aniInfo[1].split(' ')[1].split('(')[1]

        # if epiTotal == '?':
        #     print(0)
              
        
        # # consentrate
        airDate = aniInfo[2].split(' - ')[0]
        airEnd = aniInfo[2].split(' - ')[1].split('\n')[0]
        
        # ## URL Refense
        ref = elemment.find('div', class_= 'hoverinfo').get('id').split('info')[1]

        #-------------------------------

        status, premiered, broadcast, producers,studio, source, duration, ageRating, genreArray= more_detail(ref)
        
        #-------------------------------

        # Appending to Lists
        titleListnew.append(name.text)          # correct
        ratingListnew.append(float(score))      # correct
        typeListnew.append(aniType)             # correct
        episodeListnew.append((epiTotal))    # correct
        startDateListnew.append(airDate)
        endDateListnew.append(airEnd)          # correct
        membersListnew.append(int(members))
        refenceListnew.append(int(ref))
        
        ###--Exta info lists ---
        statusList.append(status)
        premieredList.append(premiered)
        broadcastList.append(broadcast)
        producerList.append(producers)
        studioList.append(studio)
        sourceList.append(source),
        durationList.append(duration)
        ageRatingList.append(ageRating)
        genreList.append(genreArray)

        # print(epiTotal)
        # print(type(ref))


### Making Dataframe with the scraped data

In [ ]:
multiPage = pd.DataFrame(data={'Rank': range(1,len(titleListnew)+1),
                               'Title':titleListnew,
                                'Rating': ratingListnew,
                                'Type': typeListnew,
                                'AirDate': startDateListnew,
                                'EndDate': endDateListnew,
                                "TotalEpi": episodeListnew,
                                'Members' : membersListnew,
                                'MAL Ref': refenceListnew,
                                
                                'Status' : statusList,                            
                                'Premiered' : premieredList,                            
                                'Broadcast' : broadcastList,
                                'Studio' : studioList,                        
                                'Duration' : durationList,                        
                                'Age Ratingn' : ageRatingList,
                                'Genres' : genreList,
                                'Producer' : producerList,                            
                                'Source' : sourceList                        
                                # 'Premiered' : List,                            
                                })

multiPage.set_index('Rank').reset_index()
# multiPage[['AirDate', 'EndDate']] = multiPage[['AirDate', 'EndDate']].apply(pd.to_datetime)
# multiPage['AirDate'] = (pd.to_datetime(multiPage['AirDate']))


multiPage.head()


,Rank,Title,Rating,Type,AirDate,EndDate,TotalEpi,Members,MAL Ref,Status,Premiered,Broadcast,Studio,Duration,Age Ratingn,Genres,Producer,Source
0,1,Fullmetal Alchemist: Brotherhood,9.11,TV,Apr 2009,Jul 2010,64,3094813,5114,Finished Airing,Spring 2009,Sundays,Bones,24,R,"[Action, Action, Adventure, Adventure, Drama, ...",Aniplex,Manga
1,2,Bleach: Sennen Kessen-hen,9.10,TV,Oct 2022,Dec 2022,13,393619,41467,Finished Airing,Fall 2022,Tuesdays,Pierrot,24,R,"[Action, Action, Adventure, Adventure, Fantasy...",TV Tokyo,Manga
2,3,Steins;Gate,9.08,TV,Apr 2011,Sep 2011,24,2383691,9253,Finished Airing,Spring 2011,Wednesdays,White Fox,24,PG-13,"[Drama, Drama, Sci-Fi, Sci-Fi, Suspense, Suspe...",Frontier Works,Visual novel
3,4,Gintama°,9.07,TV,Apr 2015,Mar 2016,51,579182,28977,Finished Airing,Spring 2015,Wednesdays,Bandai Namco Pictures,24,PG-13,"[Action, Action, Comedy, Comedy, Sci-Fi, Sci-Fi]",TV Tokyo,Manga
4,5,Kaguya-sama wa Kokurasetai: Ultra Romantic,9.07,TV,Apr 2022,Jun 2022,13,761052,43608,Finished Airing,Spring 2022,Saturdays,A-1 Pictures,23,PG-13,"[Comedy, Comedy, Romance, Romance]",Aniplex,Manga


### saving to csv for easier analysis

In [ ]:
multiPage.to_excel('multiPages.xlsx')